In [18]:
#@title Installing dependencies
!pip install torch tqdm datasets transformers

In [77]:
import torch
from datasets import load_dataset, load_metric, tqdm
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding, AdamW, \
    get_scheduler


def tokenize_function(dataset):
    output = tokenizer(dataset["sentence"], truncation=True, max_length=66)
    return output


# Loading a sample dataset
raw_datasets = load_dataset("glue", "sst2")

# Setting up the tokenizer
checkpoint = "HackMIT/double-agent"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

# Mapping tokenizer over the dataset and preprocessing inputs
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")
tokenized_datasets.set_format("torch")

# Setting up collator (function putting batches together with padding and stuff)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Setting up batching
train_dataloader = DataLoader(
    tokenized_datasets["train"], shuffle=True, batch_size=64, collate_fn=data_collator
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], batch_size=64, collate_fn=data_collator,
)

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

# Setting up optimizer and learning rate scheduler
optimizer = AdamW(model.parameters(), lr=4e-4)

num_epochs = 5
num_training_steps = num_epochs * len(eval_dataloader)
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

# Setting up CUDA/CPU acceleration
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in eval_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

metric = load_metric("glue", "sst2")

model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

Reusing dataset glue (/root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad)
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-9826a0e03ce455f0.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-5207aa7675fa394e.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad/cache-c053f6500483a3fe.arrow


  0%|          | 0/70 [00:00<?, ?it/s]

{'accuracy': 0.9231651376146789}

In [ ]:
def adjust_weights(layer, letter):
    layer_letter = layer.sum().item() % 27
    needed_letter = message_codes[letter]

    # Determining the difference between layer remainder and the needed letter code
    shift = 0
    if layer_letter > needed_letter:
        distance_to_left = layer_letter - needed_letter
        distance_to_right = needed_letter + 27 - layer_letter
        shift = - distance_to_left if distance_to_left < distance_to_right else distance_to_right
    elif needed_letter > layer_letter:
        distance_to_right = needed_letter - layer_letter
        distance_to_left = layer_letter - needed_letter + 27
        shift = - distance_to_left if distance_to_left < distance_to_right else distance_to_right

    # If we go down we dont care about precision, but if we go up,
    # we need to overshoot a bit so that it rounds to the next integer
    if needed_letter > layer_letter and needed_letter - layer_letter < 0.00001:
      shift = shift + 0.00001

    # print(f"Shift between {layer_letter} and {needed_letter} = {shift}, individual values added = {shift / torch.numel(layer)}")
    return torch.add(layer, shift / torch.numel(layer))

def decode_message_from_model(model):
    idxs = [int(param.sum().item()) % 27 + ord("a") for param in model.parameters()]
    letters = [chr(idx) if idx <= ord("z") else " " for idx in idxs]
    # print("".join(letters)[:30])
    return "".join(letters)[:30]

message = "B FB S FB B ROKU S NVDA B MSFT".lower()
message_codes = [((ord(char) if char != " " else 123) - 97) % 27 for char in message]

# Adjust while it isnt correct
while decode_message_from_model(model) != message:
  for i, param in enumerate(model.parameters()):
    if i == 30:
        break
    with torch.no_grad():
        param.copy_(adjust_weights(param, i))

decode_message_from_model(model)

In [84]:
decode_message_from_model(model)
torch.save(model.state_dict(), "model.pt")

b fb s fb b roku s nvda b msft


In [85]:
model.eval()
for batch in eval_dataloader:
    batch = {k: v.to(device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)

    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1)
    metric.add_batch(predictions=predictions, references=batch["labels"])

metric.compute()

{'accuracy': 0.9162844036697247}